### Part 1 - The ProShares ETF Product

**1.1**

- Alternative Asset ETFs - ETFs that invest in non-traditional asset classes, such as real assets, or private equity.

- Alternative Strategy ETFs - ETF that apply non-traditional investment strategies, such as long/short equity, multi-asset, or risk-parity strategies.

**1.2**

- Two measures by which hedge funds are an attractive investment: 

- Main Benefits of investing in hedge funds via ETF instead of directly:

**1.3**

- ??

**1.4**

- ??